In [ ]:
!pip install ultralytics opencv-python-headless


In [ ]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
from collections import defaultdict

# Load model (you can use yolov8n.pt or yolov8s.pt depending on speed vs accuracy)
model = YOLO("yolov8s.pt")

# COCO Class List
class_list = model.model.names


In [ ]:
# Simple Centroid-based tracker
class SimpleTracker:
    def __init__(self):
        self.center_points = {}
        self.id_count = 0

    def update(self, objects_rect):
        objects_bbs_ids = []
        new_center_points = {}

        for rect in objects_rect:
            x1, y1, x2, y2= rect
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            # Match with existing points
            same_object_detected = False
            for id, (px, py) in self.center_points.items():
                dist = ((cx - px)**2 + (cy - py)**2)**0.5
                if dist < 25:
                    new_center_points[id] = (cx, cy)
                    objects_bbs_ids.append([x1, y1, x2, y2, id])
                    same_object_detected = True
                    break

            if not same_object_detected:
                new_center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x1, y1, x2, y2, self.id_count])
                self.id_count += 1

        self.center_points = new_center_points.copy()
        return objects_bbs_ids

# Initialize tracker
tracker = SimpleTracker()


In [ ]:
import cv2

cap = cv2.VideoCapture("newvideo.mp4")
frame_width = 640
frame_height = 360

out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'XVID'), 20, (frame_width, frame_height))





In [ ]:
def get_vehicle_category(class_name):
    if class_name in ['bicycle', 'motorcycle']:
        return '2-wheeler'
    elif class_name in ['car', 'taxi', 'pickup', 'suv']:
        return '4-wheeler'
    elif class_name in ['bus', 'truck']:
        return 'heavy vehicle'
    elif class_name == 'person':
        return 'person'
    else:
        return None


In [ ]:
from google.colab import files
files.download('output.avi')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (frame_width, frame_height))

    results = model.predict(frame, verbose=False)
    boxes = results[0].boxes.data.cpu().numpy()

    detections = []
    # Store detections with category labels for tracking
    detections_with_labels = []
    for box in boxes:
        x1, y1, x2, y2, _, class_id = map(int, box)
        label = get_vehicle_category(class_list[class_id])
        if label:
            detections.append([x1, y1, x2, y2])
            # Add label to detections
            detections_with_labels.append([x1, y1, x2, y2, label])

    tracked = tracker.update(detections)

    # Use detections_with_labels to get label for each tracked object
    for obj in tracked:
        x1, y1, x2, y2, obj_id = obj

        # Find corresponding label
        label = None
        for det in detections_with_labels:
            dx1, dy1, dx2, dy2, det_label = det
            if abs(x1 - dx1) < 20 and abs(y1 - dy1) < 20:  # Check for close match
                label = det_label
                break

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        # Display label and object ID
        cv2.putText(frame, f"{label} {obj_id}", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,0), 1)

    out.write(frame)

cap.release()
out.release()